In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn 

In [4]:
df = pd.read_csv("S06-hw-dataset-04.csv")
df.head()
df.info()
df.describe(include="all")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 62 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      25000 non-null  int64  
 1   f01     25000 non-null  float64
 2   f02     25000 non-null  float64
 3   f03     25000 non-null  float64
 4   f04     25000 non-null  float64
 5   f05     25000 non-null  float64
 6   f06     25000 non-null  float64
 7   f07     25000 non-null  float64
 8   f08     25000 non-null  float64
 9   f09     25000 non-null  float64
 10  f10     25000 non-null  float64
 11  f11     25000 non-null  float64
 12  f12     25000 non-null  float64
 13  f13     25000 non-null  float64
 14  f14     25000 non-null  float64
 15  f15     25000 non-null  float64
 16  f16     25000 non-null  float64
 17  f17     25000 non-null  float64
 18  f18     25000 non-null  float64
 19  f19     25000 non-null  float64
 20  f20     25000 non-null  float64
 21  f21     25000 non-null  float64
 22

,id,f01,f02,f03,f04,f05,f06,f07,f08,f09,...,f52,f53,f54,f55,f56,f57,f58,f59,f60,target
count,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,...,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.00000
mean,12500.500000,-0.000386,-0.004872,0.003202,0.335329,-0.000563,-0.010118,0.001182,0.957385,-0.004658,...,-0.166251,-3.387506,1.749793,-0.013017,-0.001383,0.893365,-0.909479,0.000570,-0.000754,0.04920
std,7217.022701,1.001623,0.995606,1.004367,3.207537,0.993965,1.002172,2.432162,1.830223,1.013470,...,4.234741,4.331576,5.318660,1.001594,0.996409,2.445185,1.962618,0.994320,0.997167,0.21629
min,1.000000,-4.370993,-4.087073,-4.103875,-13.249937,-4.118778,-3.895974,-8.883224,-8.132548,-4.068933,...,-20.021141,-18.332290,-20.336666,-4.349216,-4.119472,-9.508509,-7.919287,-4.038312,-3.812255,0.00000
25%,6250.750000,-0.680165,-0.675100,-0.675426,-1.750048,-0.669764,-0.674374,-1.647977,-0.217260,-0.688278,...,-2.897904,-6.278403,-1.775889,-0.689962,-0.676191,-0.735473,-2.226959,-0.666367,-0.665861,0.00000
50%,12500.500000,0.001859,-0.000247,0.013272,0.403483,-0.001309,-0.005994,-0.011349,0.963009,0.000414,...,-0.396946,-3.462072,1.931851,-0.020933,-0.004193,0.888535,-0.923354,0.004381,0.002420,0.00000
75%,18750.250000,0.679702,0.659523,0.683437,2.486453,0.672299,0.652629,1.658680,2.167758,0.681040,...,2.344956,-0.578540,5.473886,0.661300,0.673722,2.516790,0.395648,0.666474,0.665918,0.00000
max,25000.000000,4.208888,3.984564,3.793442,15.288250,4.020733,4.279607,9.538525,9.321099,4.261349,...,20.717964,18.818764,20.688069,4.338337,3.902131,11.880651,6.778980,3.834922,4.012639,1.00000


In [5]:
df.isna().sum().sort_values(ascending=False)

# df["target"].value_counts()

id        0
f01       0
f02       0
f03       0
f04       0
         ..
f57       0
f58       0
f59       0
f60       0
target    0
Length: 62, dtype: int64

In [6]:
X = df.drop(columns=["target", "id"], errors = "ignore")
y = df["target"]

X.shape, y.shape

((25000, 60), (25000,))

In [7]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
    X, 
    y,
    test_size = 0.25,
    random_state = 42,
    stratify = y
)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((18750, 60), (6250, 60), (18750,), (6250,))

мы фиксируем случайность разбиения, чтобы при каждом запуске получались одинаковые train/test. это нужно для правильного сравнения моделей и воспроизводимости результата

стратификация важна, потому что она сохраняет пропорции классво в train и test. без этого можно случайно получить неправильный test, и метрики будут неверными

In [8]:

def calc_metrics(y_true, y_pred):
    return {
        "accuracy": sklearn.metrics.accuracy_score(y_true, y_pred),
        "precision": sklearn.metrics.precision_score(y_true, y_pred, zero_division = 0), 
        "recall": sklearn.metrics.recall_score(y_true, y_pred, zero_division = 0), 
        "f1": sklearn.metrics.f1_score(y_true, y_pred, zero_division = 0), 
    }

def print_metrics(title, y_true, y_pred):
    mets = calc_metrics(y_true, y_pred)
    print(title)
    for i, j in mets.items():
        print(i, j)



In [9]:
# DummyClassifier

dummy = sklearn.dummy.DummyClassifier(strategy="most_frequent", random_state=42)
dummy.fit(X_train, y_train)

y_pred_dummy = dummy.predict(X_test)
print_metrics("dummy (most_frequent)", y_test, y_pred_dummy)

dummy (most_frequent)
accuracy 0.95088
precision 0.0
recall 0.0
f1 0.0


dummy - это базовая нулевая модель, которая вообще не учится и выбирает самый частый класс. любая модель должна быть лучше этой глупой модели.

In [13]:
# LogisticRegression

lr_pipe = sklearn.pipeline.Pipeline(
    steps = [
        ("scaler", sklearn.preprocessing.StandardScaler()),
        ("model", sklearn.linear_model.LogisticRegression(max_iter=1000, random_state=42)),
    ]
)

lr_pipe.fit(X_train, y_train)
y_pred_lr = lr_pipe.predict(X_test)

print_metrics("log res + standart scaler", y_test, y_pred_lr)

log res + standart scaler
accuracy 0.96272
precision 0.9111111111111111
recall 0.2671009771986971
f1 0.41309823677581864


логистическая регрессия - довольно быстрый и сильный baseline. масштабирование признаков нужно, чтобы модель работала при разных порядков величин у фич.

In [15]:
cv = sklearn.model_selection.StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

def evaluate_on_test(name, model):
    y_pred = model.predict(X_test)
    m = calc_metrics(y_test, y_pred)
    return {"model": name, **m}
result = []
best_model = {}

In [16]:
dectree = sklearn.tree.DecisionTreeClassifier(random_state=42)

dectree_grid = {
    "max_depth": [3, 5, 7], 
    "min_samples_leaf": [5, 10, 20]
}

dectree_search = sklearn.model_selection.GridSearchCV(
    estimator = dectree, 
    param_grid = dectree_grid, 
    scoring = "f1", 
    cv = cv, 
    n_jobs = 1
)

dectree_search.fit(X_train, y_train)
best_dectree = dectree_search.best_estimator_

print("DecTree best params:", dectree_search.best_params_)
best_model["DecTree"] = best_dectree

DecTree best params: {'max_depth': 7, 'min_samples_leaf': 10}


In [18]:
randforest = sklearn.ensemble.RandomForestClassifier(random_state=42)

randforest_grid = {
    "n_estimators": [100, 200], 
    "max_depth": [8, 12], 
    "max_features": ["sqrt"],
    "min_samples_leaf": [5, 10], 
}

randforest_search = sklearn.model_selection.GridSearchCV(
    estimator = randforest, 
    param_grid = randforest_grid,
    scoring = "f1",
    cv = cv, 
    n_jobs = 1,
)

randforest_search.fit(X_train, y_train)
best_randforest = randforest_search.best_estimator_

print("RandForest best params: ", randforest_search.best_params_)
best_model["RandomForest"] = best_randforest

KeyboardInterrupt: 

In [ ]:
# AdaBoost 
ada = sklearn.ensemble.AdaBoostClassifier(random_state = 42)

ada_grid = {
    "n_estimators": [50, 100, 200],
    "learning_rate": [0.1, 0.3, 1.0],
}

ada_search = sklearn.model_selection.GridSearchCV(
    estimator = ada, 
    param_grid = ada_grid, 
    scoring = "f1",
    cv = cv,
    n_jobs = 1,
)

ada_search.fit(X_train, y_train)
best_ada = ada_search.best_estimator_

print("AdaBoost best:", ada_search.best_params_)
best_model["AdaBoost"] = best_ada

In [ ]:
lr_base = sklearn.pipeline.Pipeline(
    steps=[
        ("scaler", sklearn.preprocessing.StandardScaler()),
        ("model", sklearn.linear_model.LogisticRegression(max_iter=1000, random_state=42)),]
)

stack = sklearn.ensemble.StackingClassifier(
    estimators=[
        ("lr", lr_base),
        ("dt", best_dectree),
        ("rf", best_randforest),
        ("ada", best_ada),
    ],
    final_estimator=sklearn.linear_model.LogisticRegression(max_iter=1000, random_state=42),
    cv=cv,
    n_jobs=1,
    stack_method="predict_proba", 
)

stack.fit(X_train, y_train)
best_model["Stacking"] = stack

In [ ]:
for name, model in best_model.items():
    result.append(evaluate_on_test(name, model))

results_df = pd.DataFrame(result).sort_values(by="f1", ascending=False)
results_df